In [96]:
!pip install opencv-python==4.1.2.30

In [97]:
!pip install tensorflow==1.14.0

In [1]:

# It loads the classifier uses it to perform object detection on a webcam feed.
# It draws boxes and scores around the objects of interest in each frame from
# the webcam.

# Import packages
import os
import cv2
import numpy as np
import tensorflow as tf
import sys
from utils import label_map_util
from utils import visualization_utils as vis_util

In [2]:
detection_graph = tf.Graph()


# Import utilites
# from utils import label_map_util

# Name of the directory containing the object detection module we're using
TRAINED_MODEL_DIR = 'frozen_graphs'

# Path to frozen detection graph .pb file, which contains the model that is used
# for object detection.
PATH_TO_CKPT = TRAINED_MODEL_DIR + '/frozen_inference_graph.pb'
print(PATH_TO_CKPT)
# Path to label map file
PATH_TO_LABELS = TRAINED_MODEL_DIR + '/labelmap.pbtxt'

frozen_graphs/frozen_inference_graph.pb


In [3]:
# Number of classes the object detector can identify
NUM_CLASSES = 1

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

print(category_index)

W0611 08:50:22.249634 27568 deprecation_wrapper.py:119] From C:\Users\ragav\Downloads\helmet_detection_final\utils\label_map_util.py:133: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



{1: {'id': 1, 'name': 'with_helmet'}}


In [4]:
print("> ====== Loading frozen graph into memory")
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)
    print(">  ====== Inference graph loaded.")

> ====== Loading frozen graph into memory
>  ====== Inference graph loaded.


In [46]:
# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')


# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')


In [47]:
print(image_tensor)

Tensor("image_tensor:0", shape=(?, ?, ?, 3), dtype=uint8)


In [48]:
detection_boxes

<tf.Tensor 'detection_boxes:0' shape=<unknown> dtype=float32>

In [49]:
image = cv2.imread('image4.jpg')


In [50]:
image.size

3728400

In [51]:
image = cv2.resize(image, (800,700))
image_expanded = np.expand_dims(image, axis=0)
print(image.size)
print(image.shape)

1680000
(700, 800, 3)


In [52]:
class_names_mapping = {
            1: "with_helmet",2:"motorcycle"}

In [53]:
(boxes, scores, classes, num) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={image_tensor: image_expanded})

In [41]:
result = scores.flatten()
res = []
for idx in range(0, len(result)):
    if result[idx] > .40:
        res.append(idx)

top_classes = classes.flatten()
# Selecting class 2 and 3
#top_classes = top_classes[top_classes > 1]
res_list = [top_classes[i] for i in res]

class_final_names = [class_names_mapping[x] for x in res_list]
top_scores = [e for l2 in scores for e in l2 if e > 0.30]

In [62]:
scores.flatten()

array([9.9994969e-01, 9.9989414e-01, 1.9424933e-03, 1.8327813e-04,
       5.6751942e-05, 3.6621797e-05, 3.2412467e-05, 3.2117146e-05,
       2.2014267e-05, 1.2585371e-05, 1.0615051e-05, 8.4403691e-06,
       7.3463711e-06, 6.9216871e-06, 6.4415744e-06, 5.6095905e-06,
       4.3096707e-06, 3.8491030e-06, 3.1485131e-06, 3.0594674e-06,
       2.8681518e-06, 2.5539284e-06, 2.4382496e-06, 2.2991226e-06,
       2.2960244e-06, 1.7156862e-06, 1.5893220e-06, 1.3152633e-06,
       1.2326932e-06, 1.1697202e-06, 1.0246614e-06, 1.0114248e-06,
       9.6525548e-07, 8.9149978e-07, 8.5885023e-07, 8.4856271e-07,
       8.3783561e-07, 7.9306528e-07, 7.1229857e-07, 7.0502728e-07,
       6.6278767e-07, 6.5945238e-07, 6.2902353e-07, 6.2260352e-07,
       5.9501838e-07, 5.7159093e-07, 5.5793481e-07, 5.3533103e-07,
       5.2216217e-07, 3.9653565e-07, 3.9184823e-07, 3.8484316e-07,
       3.8178527e-07, 3.5515615e-07, 3.3651492e-07, 3.1467820e-07,
       3.0670788e-07, 2.9830062e-07, 2.9759232e-07, 2.9219478e

In [42]:
new_scores = scores.flatten()

new_boxes = boxes.reshape(300, 4)

# get all boxes from an array
max_boxes_to_draw = new_boxes.shape[0]
# this is set as a default but feel free to adjust it to your needs
min_score_thresh = .30

In [64]:
vis_util.visualize_boxes_and_labels_on_image_array(
    image,
    np.squeeze(boxes),
    np.squeeze(classes).astype(np.int32),
    np.squeeze(scores),
    category_index,
    use_normalized_coordinates=True,
    line_thickness=6,
    min_score_thresh=0.30)
cv2.imshow('Object detector', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [44]:
cv2.imshow('Object detector', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [49]:
result = scores.flatten()
res = []
for idx in range(0, len(result)):
    if result[idx] > .40:
        res.append(idx)

top_classes = classes.flatten()
# Selecting class 2 and 3
#top_classes = top_classes[top_classes > 1]
res_list = [top_classes[i] for i in res]

class_final_names = [self.class_names_mapping[x] for x in res_list]
top_scores = [e for l2 in scores for e in l2 if e > 0.30]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [37]:
for i in classes_flat2:
    if i == 2:
        vis_util.visualize_boxes_and_labels_on_image_array(
        image,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=8,
        min_score_thresh=0.75)
#         cv2.putText(image, text, org, font, fontScale, 
#                  color, thickness, cv2.LINE_AA, False)
        cv2.putText(image, "ALERT",(0, 180),cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0,255), 2)
        playsound(r"C:\Users\ragav\OneDrive\Pictures\ineuron notes\face_\alert.wav")
        cv2.imshow('Object detector', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

KeyboardInterrupt: 

In [ ]:
hjeeekjekjkjek

In [48]:
vis_util.visualize_boxes_and_labels_on_image_array(
        image,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=8,
        min_score_thresh=0.75)
    # All the results have been drawn on the frame, so it's time to display it.
cv2.imshow('Object detector', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
result = scores.flatten()
res = []
for idx in range(0, len(result)):
    if result[idx] > .40:
        res.append(idx)

In [53]:
k=np.squeeze(classes).astype(np.int32).flatten

In [54]:
k

<function ndarray.flatten>